### This notebook should contain:

1. A visualisation of what the input data looks like

2. Step by step display of the TAPS bam2read function

3. A series of graphs visualising all aspects of output, including CpG count, methylation density, base density, length distribution etc

4. A taps_bam_to_read_trimmed function with heavy logging, such that a count of each step is measured. Eg, how many reads are filtered by read.is_read1, how many have g-->A vs c-->T transition, read length, methy length, number of methylated CpGs, number of methylated non-CpGs, etc

In [1]:
import numpy as np
import pandas as pd
import os
import os.path
import pysam
from pyfaidx import Fasta
from collections import Counter
import random
import re
import codecs
import collections
import pickle
import time
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


import sys
sys.path.append('/well/ludwig/users/dyp502/read_classifier/code/sample_processing/')
from bam_processing_functions import dismir_bam_to_read_trimmed, taps_bam_to_read_trimmed

In [2]:
hg38_dir = '/well/ludwig/users/dyp502/read_classifier/hg38.fa'
genes = Fasta(hg38_dir) 

In [3]:
experiments_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/"
with open(experiments_dir + 'sample_metadata_dict.pickle', 'rb') as handle:
    sample_metadata_dict = pickle.load(handle)
sample_metadata_df = pd.read_csv(experiments_dir + "sample_metadata_df.csv")

## Inspect input data

- Test on TAPS filtered Healthy_cfDNA .bam, and TAPS tissue .bam

- Visualise output in read_df

- Compare to output for DISMIR reads

### Open bam file and inspect single line

Our input is a pysam object representing the bam file.

In [4]:
processed_bam_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/filtered_bams/dismir_10x_switching_regions/split_2/"
healthy_cfdna = list(sample_metadata_df.loc[(sample_metadata_df['status'].isin(['healthy_cfdna']))]['samples'])
# sample = 'CTR134'
# sample = 'N2L_trimmed_CD'
sample = 'Healthy_X2875'
file = [u for u in os.listdir(processed_bam_dir) if sample in u][0]
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')


#### Each line in our input object is a read

The input object is an iterable, where each iteration yields 1 read

In [5]:
for read in input:
    print(read.to_string().replace('\t','\n'))
    break

A00711:163:H5F5HDSXY:4:2377:3504:26913
147
chr1
1178356
60
150M
=
1178341
-165
TTGAGACCAGCCTGACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATTGCTTGAACCTGGGAGGCAGAGGTTGCAGTG
FF:FFFFFFFFFF,FFFFFF:F,FFFFF:FFFFFFF:FFFFFFF:FFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
MC:Z:151M
MD:Z:119C30
PG:Z:MarkDuplicates.G
NM:i:1
AS:i:145
XS:i:96


#### Each read has attributes

In [6]:
chrom = read.reference_name
start = str(read.reference_start)
end = str(read.reference_end)
read_sequence = read.query_sequence
print("\n".join([chrom,start,end,read_sequence]))

chr1
1178355
1178505
TTGAGACCAGCCTGACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATTGCTTGAACCTGGGAGGCAGAGGTTGCAGTG


## bam2read conversion steps

The cells below explain each step of the TAPS bam2read conversion function.

This function loops through each read in (TAPS-sequenced) __BWA aligned BAM files__ and outputs the sequence and methylation positions of each read.  

### Assigning methylation

- TAPS reads were aligned using bwa, which is not methylation aware so we don't have a methylation string (unlike BS-Seeker2)

- Need to call methylation by __finding differences between reference and read sequences__. Specifically:

    - On OT strand: c-->T transition
    - On OB strand: g-->A transition


#### 1. C--T conversion depends on whether the read is OT or OB

OT flag=99, OB flag=83

In [7]:
strand_mutation_mapping = {}
strand_mutation_mapping[83] = ['g','A']
strand_mutation_mapping[99] = ['c','T']


#### 2. Only include reads with flag == {99,83}

In [8]:
for read in input:
    if (read.is_read1) & (read.flag in [99,83]):
        methylation_base_change = strand_mutation_mapping[read.flag]
        reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
        read_sequence = read.query_sequence # get sequence of read
        name = read.reference_name
        break
        
print(read.flag)
print(methylation_base_change)
print(reference_sequence)
print(read_sequence)
print(name)

99
['c', 'T']
ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATcGCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATcGCCCC
ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATTGCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATTGCCCC
chr1


#### Only include chromosomes 1-22

In [9]:
chroms = ["chr"+str(i) for i in range(1,23)]
for read in input:
    if (read.is_read1) & (read.flag in [99,83]): # Only interested in read 1 from each read pair
        methylation_base_change = strand_mutation_mapping[read.flag]
        reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
        read_sequence = read.query_sequence # get sequence of read
        name = read.reference_name
        if name in chroms:
            start = str(read.reference_start)
            end = str(read.reference_end)
            # read_length = len(read_sequence)

            # Loop through each position in get_aligned_pairs tuple
            # If c-->T (or g-->A for OB strand), then add "1" to meth_list, otherwise add "0"
            meth_str = ""
            methylation = 0 # track whether each read is methylated or not 
            read_aligned_pairs = read.get_aligned_pairs(with_seq=True)
            length_count = 0
            break

print(read.flag)
print(methylation_base_change)
print(reference_sequence)
print(read_sequence)
print(name)
print(start)
print(end)
# print(read_aligned_pairs)

83
['g', 'A']
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCgCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCgCCCCA
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCACTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCACCCCA
chr1
1178370
1178521


### Practical examples: assigning read methylation to BWA aligned reads
    
- When there are no insertions or deletions, read and reference string have same length, so we can easily spot methylation. However we want to also account for possible insertions and deletions.

- Difficulty is indexing the reference and read sequence to ensure we're always referring to same relative position on these two sequences. Insertions and deletions offset the balance.

#### Simple case: take example read above

- The reference and read sequence are the same length. 

- In this case assigning methylation is easy, we simply loop through the reference sequence, and record position of all c->T (if OT) or g->A (if OB) transitions.

In [10]:
print(reference_sequence)
print(read_sequence)

print(len(reference_sequence))
print(len(read_sequence))

print(f"\nRead has flag {read.flag}, so look for {strand_mutation_mapping[read.flag]} transitions")


CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCgCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCgCCCCA
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCACTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCACCCCA
151
151

Read has flag 83, so look for ['g', 'A'] transitions


In [11]:
methylation_base_change = strand_mutation_mapping[read.flag]

meth_str = ""
for k,ref_base in enumerate(reference_sequence):
    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
        print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
        meth_str += "1"
    else:
        meth_str += "0"
        
print('\nResults:\n\n')
print(reference_sequence)
print(read_sequence)
print(meth_str)

g to A transition at position 105, so methylation
g to A transition at position 145, so methylation

Results:


CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCgCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCgCCCCA
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCACTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCACCCCA
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000100000


#### We must also check methylation is in CpG context

- On OT strand we expect reference base to be C, followed by a G. So reference_sequence[k] == c, reference_sequence[k+1] == G.

- On OB strand we expect reference base to be G, __preceded by a C__. So reference_sequence[k] == g, reference_sequence[k-1] == C.

- In practice, can we just check one either side of the reference base, ie reference_sequence[k-1:k+2]

In [12]:
strand_cpg_check = {}
strand_cpg_check[83] = 'Cg'
strand_cpg_check[99] = 'cG'

In [13]:
methylation_base_change = strand_mutation_mapping[read.flag]


meth_str = ""
for k,ref_base in enumerate(reference_sequence):
    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
        if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
            print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
            print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
            
            meth_str += "1"
    else:
        meth_str += "0"
        
print('\nResults:\n\n')
print(reference_sequence)
print(read_sequence)
print(meth_str)

g to A transition at position 105, so methylation
Cg is in CgC, so CpG site
g to A transition at position 145, so methylation
Cg is in CgC, so CpG site

Results:


CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCgCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCgCCCCA
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCACTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCACCCCA
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000100000


#### Edge cases? Can't check CpG context if CpG is at very end or very start of sequence

- DISMIR ignore this problem, assuming if 'C' at end of sequence then CpG methylation

- To improve upon this, we need to obtain CpG context outside of reference sequence. 

- There are only two cases when this would arise:
    - c->G at end of sequence, when k==len(reference_sequence-1) 
    - g->A transition at start of sequence, when k==0

In [14]:
methylation_base_change = strand_mutation_mapping[read.flag]


meth_str = ""
for k,ref_base in enumerate(reference_sequence):
    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
        if (k==len(reference_sequence)-1) and (read.flag==99):
            print('Need additional context to check if CpG methylation')
        elif (k==0) and (read.flag==83):
            print('Need additional context to check if CpG methylation')

        elif strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
            print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
            print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
            
            meth_str += "1"
    else:
        meth_str += "0"
        
print('\nResults:\n\n')
print(reference_sequence)
print(read_sequence)
print(meth_str)

g to A transition at position 105, so methylation
Cg is in CgC, so CpG site
g to A transition at position 145, so methylation
Cg is in CgC, so CpG site

Results:


CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCgCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCgCCCCA
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCACTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCACCCCA
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000100000


### Check for edge cases

- To check CpG context at very start or end of read, can use pyfaidx genex library


In [15]:
read_encodings = []
read_dict = collections.defaultdict(list)
read_object_dict = {}
save = False
trimmed = False
read_count = 0 
bam_metrics = Counter()
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

for read in input:
    if (read.is_read1) & (read.flag in [99,83]):
        bam_metrics['read_1_and_99_83'] +=1
        if ('I' not in read.cigarstring) & ('D' not in read.cigarstring):
            bam_metrics['no_mutation_deletion'] +=1
#         if 'M' in read.cigarstring:
     # Only interested in read 1 from each read pair
            methylation_base_change = strand_mutation_mapping[read.flag]
            reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
            read_sequence = read.query_sequence # get sequence of read
            name = read.reference_name
            if name in chroms:
                bam_metrics['in_chr_1_22'] +=1
                start = str(read.reference_start)
                end = str(read.reference_end)
                # Loop through each position in get_aligned_pairs tuple
                # If c-->T (or g-->A for OB strand), then add "1" to meth_list, otherwise add "0"
                methylation_base_change = strand_mutation_mapping[read.flag]
                meth_str = ""
                for k,ref_base in enumerate(reference_sequence):
                    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
                        bam_metrics['methylation_change'] +=1
                        print("Correct change observed, now just need to check CpG context")
                        
                        if (k==len(reference_sequence)-1) and (read.flag==99): # if methylation on final base of OT, need to check that subsequent base is G
                            print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1
                            else:
                                print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1

                        elif (k==0) and (read.flag==83): # if methylation on very first base of OB, need to check that preceding base was C
                            print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1


                        else:
                            if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
                                print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
                                print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
                                context = genes[name][int(start)+k-1:int(start)+k+2] 
                                print(context)
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                meth_str += "0"
                                print('Non-CpG methylation')
                                bam_metrics['non_cpg_methylation'] +=1

                    else:
                        meth_str += "0"
                        
            

            print(f'\n\nRead {read_count}:\n')
            print(name + ": " + start + '-' + end)
            print(read.flag)
            print(reference_sequence)
            print(read_sequence)
            print(meth_str)
            read_count +=1
            
            
            if read_count > 100:
                break
                
bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])


Correct change observed, now just need to check CpG context
c to T transition at position 105, so methylation
cG is in TcG, so CpG site
tcg
Correct change observed, now just need to check CpG context
c to T transition at position 145, so methylation
cG is in TcG, so CpG site
tcg


Read 0:

chr1: 1178369-1178520
99
ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATcGCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATcGCCCC
ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATTGCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATTGCCCC
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000100000
Correct change observed, now just need to check CpG context
g to A transition at position 105, so methylation
Cg is in CgC, so CpG site
cgc
Correct change observed, now just need to check CpG context
g to A transition at positio

c to T transition at position 84, so methylation
cG is in AcG, so CpG site
ACG
Correct change observed, now just need to check CpG context
c to T transition at position 95, so methylation
cG is in CcG, so CpG site
CCG
Correct change observed, now just need to check CpG context
c to T transition at position 99, so methylation
cG is in AcG, so CpG site
ACG
Correct change observed, now just need to check CpG context
c to T transition at position 117, so methylation
cG is in AcG, so CpG site
ACG
Correct change observed, now just need to check CpG context
c to T transition at position 130, so methylation
cG is in GcG, so CpG site
GCG


Read 40:

chr1: 1178950-1179101
99
CCAGGGAGGACATGCCcGGCAGAGACAGGGTGCAGAGACAGCCTcGCCCCcACAcGGcGCCAGGCCCCACAGCTGCCACAGAGAcGGGAGCCAGCcGCAcGCAGAGGCTCAAGCCCAcGGCCCTGGGAGGcGCCCTTTCCTGGGCAGGGTG
CCAGGGAGGACATGCCTGGCAGAGACAGGGTGCAGAGACAGCCTTGCCCCTACATGGTGCCAGGCCCCACAGCTGCCACAGAGATGGGAGCCAGCTGCATGCAGAGGCTCAAGCCCATGGCCCTGGGAGGTGCCCTTTCCTGGGCAGGGTG
0000000000000000100000

g to A transition at position 14, so methylation
Cg is in CgC, so CpG site
CGC
Correct change observed, now just need to check CpG context
g to A transition at position 16, so methylation
Cg is in CgA, so CpG site
CGA
Correct change observed, now just need to check CpG context
g to A transition at position 22, so methylation
Cg is in CgG, so CpG site
CGG
Correct change observed, now just need to check CpG context
g to A transition at position 93, so methylation
Cg is in CgC, so CpG site
CGC


Read 53:

chr1: 1219016-1219167
83
GGACACAGCCACCCgCgAGACCgGGGCCCCACCCTCCAGGATTCACACGTCCCCAGAACATGGCCCAGCCCCTAAGACACAGCCTGGACCCCCgCCAAGACATAACCCAGAGCCTCCCACAGGCCTGATCCCCAAAGGCATAGCTTGGACC
GGACACAGCCACCCACAAGACCAGGGCCCCACCCTCCAGGATTCACACGTCCCCAGAACATGGCCCAGCCCCTAAGACACAGCCTGGACCCCCACCAAGACATAACCCAGAGCCTCCCACAGGCCTGATCCCCAAAGGCATAGCTTGGACC
0000000000000010100000100000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000
Correct cha

Correct change observed, now just need to check CpG context
g to A transition at position 34, so methylation
Cg is in Cgg, so CpG site
CGG
Correct change observed, now just need to check CpG context
Non-CpG methylation
Correct change observed, now just need to check CpG context
g to A transition at position 37, so methylation
Cg is in CgC, so CpG site
CGC
Correct change observed, now just need to check CpG context
g to A transition at position 40, so methylation
Cg is in CgC, so CpG site
CGC
Correct change observed, now just need to check CpG context
g to A transition at position 42, so methylation
Cg is in CgG, so CpG site
CGG
Correct change observed, now just need to check CpG context
g to A transition at position 47, so methylation
Cg is in CgA, so CpG site
CGA
Correct change observed, now just need to check CpG context
g to A transition at position 69, so methylation
Cg is in CgT, so CpG site
CGT
Correct change observed, now just need to check CpG context
g to A transition at posit

### Combine everything together

In [16]:
read_encodings = []
read_dict = collections.defaultdict(list)
read_object_dict = {}
save = False
trimmed = False
read_count = 0 
bam_metrics = Counter()
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

for read in input:
    bam_metrics['total_read_count'] +=1
    name = read.reference_name

    if name in chroms:
        bam_metrics['in_chr_1_22'] +=1

        if (read.is_read1) & (read.flag in [99,83]):

            bam_metrics['read_1_and_99_83'] +=1
#         if ('I' not in read.cigarstring) & ('D' not in read.cigarstring):
#         if 'M' in read.cigarstring:
 # Only interested in read 1 from each read pair
            methylation_base_change = strand_mutation_mapping[read.flag]
            reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
            read_sequence = read.query_sequence # get sequence of read

            start = str(read.reference_start)
            end = str(read.reference_end)
            # Loop through each position in get_aligned_pairs tuple
            # If c-->T (or g-->A for OB strand), then add "1" to meth_list, otherwise add "0"
            methylation_base_change = strand_mutation_mapping[read.flag]
            
            
        
            print(f'\n\nRead {read_count}:\n')
            print(name + ": " + start + '-' + end)
            print(read.flag)
            print(reference_sequence)
            print(read_sequence)
            
        
            meth_str = ""
            bam_metrics['no_mutation_deletion'] +=1
            for k,ref_base in enumerate(reference_sequence):
                if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
                    bam_metrics['methylation_change'] +=1
                    print("Correct change observed, now just need to check CpG context")

                    if (k==len(reference_sequence)-1) and (read.flag==99): # if methylation on final base of OT, need to check that subsequent base is G
                        print('Need additional context to check if CpG methylation')
                        context = genes[name][int(start)+k-1:int(start)+k+2] 
                        if 'CG' in context:
                            print('context contains CpG so CpG methylation')
                            meth_str += "1"
                            bam_metrics['cpg_methylation'] +=1
                        else:
                            print('No CpG in context so non-CpG methylation')
                            meth_str += "0"
                            bam_metrics['non_cpg_methylation'] +=1

                    elif (k==0) and (read.flag==83): # if methylation on very first base of OB, need to check that preceding base was C
                        print('Need additional context to check if CpG methylation')
                        context = genes[name][int(start)+k-1:int(start)+k+2] 
                        if 'CG' in context:
                            print('context contains CpG so CpG methylation')
                            meth_str += "1"
                            bam_metrics['cpg_methylation'] +=1

                        else:
                            print('No CpG in context so non-CpG methylation')
                            meth_str += "0"
                            bam_metrics['non_cpg_methylation'] +=1


                    else:
                        if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
                            print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
                            print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            print(context)
                            meth_str += "1"
                            bam_metrics['cpg_methylation'] +=1

                        else:
                            meth_str += "0"
                            print('Non-CpG methylation')
                            bam_metrics['non_cpg_methylation'] +=1

                else:
                    meth_str += "0"

            if len(meth_str) != len(reference_sequence):
                print('Meth str does not match sequence so error')
                break
                
            if trimmed:
                reference_sequence = reference_sequence[5:71]
                meth_str = meth_str[5:71]
                read_sequence = read_sequence[5:71]
            if save:
                output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
            else:
                read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                read_dict['read_name'].append(read.query_name)
                read_dict['strand'].append(read.flag)
                read_dict['chr'].append(read.reference_name)
                read_dict['start_pos'].append(start)
                read_dict['ref_sequence'].append(reference_sequence.upper())
                read_dict['read_sequence'].append(read_sequence.upper())
                read_dict['meth_str'].append(meth_str)
                read_object_dict[read.query_name] = read


 
            print(meth_str)
            read_count +=1
            
            
            if read_count > 100:
                break
                
bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])




Read 0:

chr1: 1178369-1178520
99
ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATcGCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATcGCCCC
ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATTGCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATTGCCCC
Correct change observed, now just need to check CpG context
c to T transition at position 105, so methylation
cG is in TcG, so CpG site
tcg
Correct change observed, now just need to check CpG context
c to T transition at position 145, so methylation
cG is in TcG, so CpG site
tcg
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000100000


Read 1:

chr1: 1178370-1178521
83
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTGTAGTCCCAGCTACCTGGGAGGCTGAGGCAGGAGAATCgCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCTAGATCgCCCCA
CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAA

IndexError: string index out of range

### Need special treatment of Insertions and Deletions

- If a read contains an insertion or a deletion, the reference_position and read_position go out of sync

- We detect CpG methylation by comparing read sequence to reference sequence, so need this correspondence

- Need a separate method to detect methylation in the presence of insertions/deletions

- Edge cases;
     - Insertion at end of reference sequence
     - Insertion/deletion that matches CpG methylation change, eg reference at pos 27 = c, and read has T insertion at pos 27, eg Healthy_X2875, chr1:204707383


__Line by line Function Description:__


for read in input bam file
if read is in chroms 1-22, is read1, and is properly paired (has bwa flag 83 (OB) or 99 (OT)), then proceed
get read characteristics, eg start, end, reference sequence, read sequence
If c-->T (or g-->A for OB strand), then add "1" to meth_list, otherwise add "0"


meth_str must correspond exactly to __reference sequence__. Insertions or deletions can break this correspondence
If insertion: 
- read sequence 1bp longer than reference sequence
- meth_str should still follow reference sequence (and not add any value to meth_str at insertion position)

If deletion:
- Read sequence 1bp shorter than reference sequence
- meth_str should still follow reference sequence (and just add "0" at deletion position)

In [17]:
read_encodings = []
read_dict = collections.defaultdict(list)
read_object_dict = {}
save = False
trimmed = False
verbose=False
include_indels = True
read_count = 0 
bam_metrics = Counter()
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

for read in input:
    bam_metrics['total_read_count'] +=1
    name = read.reference_name
    if name in chroms:
        bam_metrics['in_chr_1_22'] +=1
        if (read.is_read1) & (read.flag in [99,83]):  # Only interested in read 1 from each read pair
            bam_metrics['read_1_and_99_83'] +=1
            methylation_base_change = strand_mutation_mapping[read.flag]
            reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
            read_sequence = read.query_sequence # get sequence of read

            start = str(read.reference_start)
            end = str(read.reference_end)
            # Loop through each position in get_aligned_pairs tuple
            methylation_base_change = strand_mutation_mapping[read.flag]
            
            
            if verbose:
                print(f'\n\nRead {read_count}:\n')
                print(name + ": " + start + '-' + end)
                print(read.flag)
                print(reference_sequence)
                print(read_sequence)
            meth_str = ""
            
            if ('I' in read.cigarstring and include_indels) or ('D' in read.cigarstring and include_indels):
                if verbose:
                    print("Insertion or deletion in read") 
                bam_metrics['insertion_or_deletion'] +=1
                read_index,ref_index = 0,0
                read_aligned_pairs = read.get_aligned_pairs(with_seq=True)
                for k,(read_pos,ref_pos,ref_base) in enumerate(read_aligned_pairs):
                    insertion=False
                    deletion=False

                    if read_pos==None: # if read_pos is none, then deletion in read at this position
                        deletion=True
                        bam_metrics['deletion'] +=1
                    elif ref_base==None: # if ref_base is none, then insertion in read at this position
                        insertion=True
                        bam_metrics['insertion'] +=1
                
                        if ref_index==len(reference_sequence): # an insertion at the end of the reference_sequence will break the indexing
                            bam_metrics['insertion_at_end_of_read'] +=1
                            if verbose:
                                print('Insertion at very end of reference sequence, skipping remaining bases on read')
                            continue # go back to the top if insertion at end of sequence, ie skip over remaining bases

                    if verbose:
                        print(k,(read_pos,ref_pos,ref_base))
                        print(f"ref_index: {ref_index}")
                        print(f"read_index: {read_index}")
                        print(reference_sequence[ref_index])
                        print(read_sequence[read_index])
                #     if not read_pos==None: # if there is no deletion (read_pos is None when deletion)
                    if (reference_sequence[ref_index] == methylation_base_change[0]) & (read_sequence[read_index] == methylation_base_change[1]): # Require either cT or gA
                        if verbose:
                            print("Correct change observed, now just need to check CpG context")
                #         if (ref_base == methylation_base_change[0]) & (read_sequence[read_pos] == methylation_base_change[1]): # Require either cT or gA
                        context = genes[name][int(start)+ref_index-1:int(start)+ref_index+2] # pyfaidx genes are zero indexed so need to shift -1
                        if 'cg' in context.seq.lower():
                            if verbose:
                                print('context contains CpG so CpG methylation')
                            methylation +=1
                            meth_str += "1"
                            bam_metrics['cpg_methylation'] +=1
                        else:
                            meth_str += "0"
                            bam_metrics['non_cpg_methylation'] +=1
                            
                    else:
                        if insertion: # if insertion, then reference_sequence becomes 1bp shorter than read sequence. meth_str must match reference_sequence, so we don't append to meth_str at insertion position
                            pass
                        else: # if deletion, then reference_sequence becomes 1bp longer than read sequence. meth_str must match reference_sequence, so we append 0 to meth_str at this position
                            meth_str += "0"

                    # insertion/deletion aware read and reference position tracker
                    # this tracks the actual position on the read and ref sequence, allowing for deletions and insertions
                    if not deletion:
                        read_index +=1 
                    if not insertion:
                        ref_index +=1 


            
            else:
                bam_metrics['no_insertion_or_deletion'] +=1
                for k,ref_base in enumerate(reference_sequence):
                    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
                        bam_metrics['methylation_change'] +=1
                        if verbose:
                            print("Correct change observed, now just need to check CpG context")

                        if (k==len(reference_sequence)-1) and (read.flag==99): # if methylation on final base of OT, need to check that subsequent base is G
                            if verbose:
                                print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                if verbose:
                                    print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1
                            else:
                                if verbose:
                                    print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1

                        elif (k==0) and (read.flag==83): # if methylation on very first base of OB, need to check that preceding base was C
                            if verbose:
                                print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                if verbose:
                                    print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                if verbose:
                                    print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1


                        else:
                            if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
                                context = genes[name][int(start)+k-1:int(start)+k+2] 
                                if verbose:
                                    print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
                                    print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
                                    print(context)
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                meth_str += "0"
                                if verbose:
                                    print('Non-CpG methylation')
                                bam_metrics['non_cpg_methylation'] +=1

                    else:
                        meth_str += "0"

            if len(meth_str) != len(reference_sequence):
                print('Meth str length does not match reference sequence length so error')
                break
                
            if trimmed:
                reference_sequence = reference_sequence[5:71]
                meth_str = meth_str[5:71]
                read_sequence = read_sequence[5:71]
            if save:
                output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
            else:
                read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                read_dict['read_name'].append(read.query_name)
                read_dict['strand'].append(read.flag)
                read_dict['chr'].append(read.reference_name)
                read_dict['start_pos'].append(start)
                read_dict['ref_sequence'].append(reference_sequence.upper())
                read_dict['read_sequence'].append(read_sequence.upper())
                read_dict['meth_str'].append(meth_str)
                read_object_dict[read.query_name] = read


 
            if verbose:
                print(meth_str)
            read_count +=1
            
            
#             if read_count > 1000:
#                 break
                
bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])



Meth str length does not match reference sequence length so error


### If InDel, need to skip that position

In [ ]:
read_encodings = []
read_dict = collections.defaultdict(list)
read_object_dict = {}
save = False
trimmed = False
verbose=False
include_indels = True
read_count = 0 
bam_metrics = Counter()
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

for read in input:
    bam_metrics['total_read_count'] +=1
    name = read.reference_name
    if name in chroms:
        bam_metrics['in_chr_1_22'] +=1
        if (read.is_read1) & (read.flag in [99,83]):  # Only interested in read 1 from each read pair
            bam_metrics['read_1_and_99_83'] +=1
            methylation_base_change = strand_mutation_mapping[read.flag]
            reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
            read_sequence = read.query_sequence # get sequence of read

            start = str(read.reference_start)
            end = str(read.reference_end)
            # Loop through each position in get_aligned_pairs tuple
            methylation_base_change = strand_mutation_mapping[read.flag]
            
            
            if verbose:
                print(f'\n\nRead {read_count}:\n')
                print(name + ": " + start + '-' + end)
                print(read.flag)
                print(reference_sequence)
                print(read_sequence)
            meth_str = ""
            
            if ('I' in read.cigarstring and include_indels) or ('D' in read.cigarstring and include_indels):
                if verbose:
                    print("Insertion or deletion in read") 
                bam_metrics['insertion_or_deletion'] +=1
                read_index,ref_index = 0,0  # insertion/deletion aware read and reference position tracker
                                            # this tracks the actual position on the read and ref sequence, allowing for deletions and insertions
                read_aligned_pairs = read.get_aligned_pairs(with_seq=True)
                for k,(read_pos,ref_pos,ref_base) in enumerate(read_aligned_pairs):
                    insertion=False
                    deletion=False
                    # If there is an indel at this position, there can be no CpG methylation so we skip it
                    if read_pos==None: # if read_pos is none, then deletion in read at this position
                        deletion=True
                        ref_index +=1
                        bam_metrics['deletion'] +=1
                        meth_str += "0" # if deletion, then reference_sequence becomes 1bp longer than read sequence. meth_str must match reference_sequence, so we append 0 to meth_str at this position
                    elif ref_base==None: # if ref_base is none, then insertion in read at this position
                        insertion=True
                        read_index +=1
                        bam_metrics['insertion'] +=1
                        # if insertion, then reference_sequence becomes 1bp shorter than read sequence. meth_str must match reference_sequence, so we don't append to meth_str at insertion position

#                         if ref_index==len(reference_sequence): # an insertion at the end of the reference_sequence will break the indexing
#                             bam_metrics['insertion_at_end_of_read'] +=1
#                             if verbose:
#                                 print('Insertion at very end of reference sequence, skipping remaining bases on read')
#                             continue # go back to the top if insertion at end of sequence, ie skip over remaining bases
                    else:
                        if verbose:
                            print(k,(read_pos,ref_pos,ref_base))
                            print(f"ref_index: {ref_index}")
                            print(f"read_index: {read_index}")
                            print(reference_sequence[ref_index])
                            print(read_sequence[read_index])
                            
                    #     if not read_pos==None: # if there is no deletion (read_pos is None when deletion)
                        if (reference_sequence[ref_index] == methylation_base_change[0]) & (read_sequence[read_index] == methylation_base_change[1]): # Require either cT or gA
                            if verbose:
                                print("Correct change observed, now just need to check CpG context")
                    #         if (ref_base == methylation_base_change[0]) & (read_sequence[read_pos] == methylation_base_change[1]): # Require either cT or gA
                            context = genes[name][int(start)+ref_index-1:int(start)+ref_index+2] # pyfaidx genes are zero indexed so need to shift -1
                            if 'cg' in context.seq.lower():
                                if verbose:
                                    print('context contains CpG so CpG methylation')
                                methylation +=1
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1
                            else:
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1

                        else:
                            meth_str += "0"
                        read_index +=1 
                        ref_index +=1 

            
            else:
                bam_metrics['no_insertion_or_deletion'] +=1
                for k,ref_base in enumerate(reference_sequence):
                    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
                        bam_metrics['methylation_change'] +=1
                        if verbose:
                            print("Correct change observed, now just need to check CpG context")

                        if (k==len(reference_sequence)-1) and (read.flag==99): # if methylation on final base of OT, need to check that subsequent base is G
                            if verbose:
                                print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                if verbose:
                                    print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1
                            else:
                                if verbose:
                                    print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1

                        elif (k==0) and (read.flag==83): # if methylation on very first base of OB, need to check that preceding base was C
                            if verbose:
                                print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                if verbose:
                                    print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                if verbose:
                                    print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1


                        else:
                            if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
                                context = genes[name][int(start)+k-1:int(start)+k+2] 
                                if verbose:
                                    print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
                                    print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
                                    print(context)
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                meth_str += "0"
                                if verbose:
                                    print('Non-CpG methylation')
                                bam_metrics['non_cpg_methylation'] +=1

                    else:
                        meth_str += "0"

            if len(meth_str) != len(reference_sequence):
                print('Meth str length does not match reference sequence length so error')
                break
                
            if trimmed:
                reference_sequence = reference_sequence[5:71]
                meth_str = meth_str[5:71]
                read_sequence = read_sequence[5:71]
            if save:
                output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
            else:
                read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                read_dict['read_name'].append(read.query_name)
                read_dict['strand'].append(read.flag)
                read_dict['chr'].append(read.reference_name)
                read_dict['start_pos'].append(start)
                read_dict['ref_sequence'].append(reference_sequence.upper())
                read_dict['read_sequence'].append(read_sequence.upper())
                read_dict['meth_str'].append(meth_str)
                read_object_dict[read.query_name] = read


 
            if verbose:
                print(meth_str)
            read_count +=1
            
            
#             if read_count > 1000:
#                 break
                
bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])



In [ ]:
log_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/logs/"
log_file = log_dir+'bam_metrics.csv'
all_bam_metrics_df = pd.read_csv(log_file)



In [ ]:
all_bam_metrics_df

In [ ]:

if log_dir:
    sample = file.split('.')[0]
    bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])
    log_file = log_dir+'bam_metrics.csv'
    if os.path.exists(log_file):
        all_bam_metrics_df = pd.read_csv(log_file)
        all_bam_metrics_df[sample] = bam_metrics_df[sample]
        all_bam_metrics_df.to_csv(log_file)
    else:
        bam_metrics_df.reset_index().to_csv(log_file,index=False)


In [ ]:
bam_metrics_df[sample]

### Insertions at end of read!

In [ ]:
read_index,ref_index = 0,0
meth_str = ""
for k,(read_pos,ref_pos,ref_base) in enumerate(read_aligned_pairs):
    insertion=False
    deletion=False

    if read_pos==None: # if read_pos is none, then deletion in read at this position
        deletion=True
        bam_metrics['deletion'] +=1
    elif ref_base==None: # if ref_base is none, then insertion in read at this position
        insertion=True
        bam_metrics['insertion'] +=1
    
        if ref_index==len(reference_sequence): # an insertion at the end of the reference_sequence will break the indexing
            continue
            if verbose:
                print('Insertion at very end of reference sequence, skipping remaining bases on read')

    if verbose:
        print(k,(read_pos,ref_pos,ref_base))
        print(f"ref_index: {ref_index}")
        print(f"read_index: {read_index}")
        print(reference_sequence[ref_index])
        print(read_sequence[read_index])
#     if not read_pos==None: # if there is no deletion (read_pos is None when deletion)
    if (reference_sequence[ref_index] == methylation_base_change[0]) & (read_sequence[read_index] == methylation_base_change[1]): # Require either cT or gA
        if verbose:
            print("Correct change observed, now just need to check CpG context")
#         if (ref_base == methylation_base_change[0]) & (read_sequence[read_pos] == methylation_base_change[1]): # Require either cT or gA
#         context = genes[name][ref_pos-1:ref_pos+2] # pyfaidx genes are zero indexed so need to shift -1
#         print(f'context a: {context}')
        context_alt = genes[name][int(start)+k-1:int(start)+k+2] 
        print(f'context b: {context_alt}')
        context_alt2 = genes[name][int(start)+ref_index-1:int(start)+ref_index+2] 
        print(f'context c: {context_alt2}')
        if 'cg' in context.seq.lower():
            if verbose:
                print('context contains CpG so CpG methylation')
            methylation +=1
            meth_str += "1"
            bam_metrics['cpg_methylation'] +=1
        else:
            meth_str += "0"
            bam_metrics['non_cpg_methylation'] +=1

    else:
        if insertion: # if insertion, then reference_sequence becomes 1bp shorter than read sequence. meth_str must match reference_sequence, so we don't append to meth_str at insertion position
            pass
        else: # if deletion, then reference_sequence becomes 1bp longer than read sequence. meth_str must match reference_sequence, so we append 0 to meth_str at this position
            meth_str += "0"
    length_count += 1

    # insertion/deletion aware read and reference position tracker
    # this tracks the actual position on the read and ref sequence, allowing for deletions and insertions
    if not deletion:
#         print('not deletion')
        read_index +=1 
    if not insertion:
#         print('not insertion')
        ref_index +=1 
        

### Try new improved get_meth_str function

In [18]:
sys.path.append('/well/ludwig/users/dyp502/read_classifier/code/sample_processing/')
from bam_processing_functions import get_meth_str

In [19]:
print(read.get_reference_sequence())
print(read.query_sequence)
print(meth_str)

CCAGATCAAAGGAAGTCAGGTGTGTTCcGGGGGGTATGGCTCTAGTGTCTCTCTCTCTCTCTCCCTCTCCCCCGTTCAATTCTCAATTCCTTCCTGTCCCCGCTGCACTATTGCTAATTTCCATAGGACTTAACCCCCCCtCCCCC
CCAGATCAAAGGAAGTCAGGTGTGTTCTGGGGGGGTATGGCTCTAGTGTCTCTCTCTCTCTCTCCCTCTCCCCCGTTCAATTCTCAATTCCTTCCTGTCCCCGCTGCACTATTGCTAATTTCCATAGGACTTAACCCCCCCGCCCCC
000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [20]:
meth_str,meC,unC,cpg_count = get_meth_str(read,genes,strand_mutation_mapping,strand_cpg_check,
                                                          verbose=True)

chr1: 204707384-204707529
99
Ref:  CCAGATCAAAGGAAGTCAGGTGTGTTCcGGGGGGTATGGCTCTAGTGTCTCTCTCTCTCTCTCCCTCTCCCCCGTTCAATTCTCAATTCCTTCCTGTCCCCGCTGCACTATTGCTAATTTCCATAGGACTTAACCCCCCCtCCCCC
Read: CCAGATCAAAGGAAGTCAGGTGTGTTCTGGGGGGGTATGGCTCTAGTGTCTCTCTCTCTCTCTCCCTCTCCCCCGTTCAATTCTCAATTCCTTCCTGTCCCCGCTGCACTATTGCTAATTTCCATAGGACTTAACCCCCCCGCCCCC
Insertion, deletion or soft clipping in read
0 (0, 204707383, 'C')


0 (0, 204707383, 'C')
ref_index: 0
read_index: 0
Context checked from 204707384:204707385: found cc
Ref to read: C-->C
1 (1, 204707384, 'C')


1 (1, 204707384, 'C')
ref_index: 1
read_index: 1
Context checked from 204707385:204707386: found ca
Ref to read: C-->C
2 (2, 204707385, 'A')


2 (2, 204707385, 'A')
ref_index: 2
read_index: 2
Context checked from 204707386:204707387: found ag
Ref to read: A-->A
3 (3, 204707386, 'G')


3 (3, 204707386, 'G')
ref_index: 3
read_index: 3
Context checked from 204707387:204707388: found ga
Ref to read: G-->G
4 (4, 204707387, 'A')


4 (4, 204707387, 'A')
r

### Edge case: meth_str needs to always be same length as reference sequence

- Find cases where this doesn't apply

In [4]:
def taps_bam_to_read_nov_22(file,savename,processed_bam_dir,dest_dir,log_dir=None,trimmed=True,save=True,read_objects=False,include_indels=True,verbose=False):

    """
    New version of TAPS bam2read converter (20/05/2022). 
    
    The previous version was outputting different length
    meth_str and reference_sequence strings. This was due to bad handling of insertions & deletions, where the
    correspondence between reference_sequence and read_sequence is lost. This correspondence is vital as it is 
    used to identify c->T and g->A transitions in each read (OT and OB strand respectively).
    

    This new function properly deals with insertions and deletions, by maintaining correspondence between read_sequence
    reference_sequence and calculated meth_str. 
    
    Outputs:
        If save==True, for each read saves:
            chr
            start
            reference_sequence
            meth_str (should always have same length as reference_sequence)
        If save==False:
            Outputs the above into a read_df, alongside which we can optionally output each read_object for manual inspection (by setting read_objects=True)
    
    """
    strand_mutation_mapping = {}
    strand_mutation_mapping[83] = ['g','A']
    strand_mutation_mapping[99] = ['c','T']
    read_object_dict = {}
    strand_cpg_check = {}
    strand_cpg_check[83] = 'Cg'
    strand_cpg_check[99] = 'cG'

    hg38_dir = '/well/ludwig/users/dyp502/read_classifier/hg38.fa'
    genes = Fasta(hg38_dir) 
    chroms = ["chr"+str(i) for i in range(1,23)]
    
    
    if save:
        output = open(dest_dir + savename, 'w')
    
    read_encodings = []
    read_dict = collections.defaultdict(list)
    read_object_dict = {}
    read_count = 0 
    bam_metrics = Counter()
    if (file[-4:]=='.bam'):
        print("converting " + file)
        input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

        for read in input:
            bam_metrics['total_read_count'] +=1
            name = read.reference_name
            if name in chroms:
                bam_metrics['in_chr_1_22'] +=1
                if (read.is_read1) & (read.flag in [99,83]):  # Only interested in read 1 from each read pair
                    bam_metrics['read_1_and_99_83'] +=1
                    meth_str,meC,unC,cpg_count = get_meth_str(read,genes,strand_mutation_mapping,strand_cpg_check,
                                                          verbose=False)
                    
                    
                    ## Define aspects of read (sequence, start etc)
                    reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
                    read_sequence = read.query_sequence # get sequence of read
                    start = str(read.reference_start)
                    end = str(read.reference_end)
                    
                    
                    if trimmed:
                        reference_sequence = reference_sequence[5:71]
                        meth_str = meth_str[5:71]
                        read_sequence = read_sequence[5:71]
                    if save:
                        output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
                    else:
                        read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                        read_dict['id'].append(read.qname)
                        read_dict['chrom'].append(read.reference_name)
                        read_dict['start'].append(start)
                        read_dict['stop'].append(end)
                        read_dict['meth_str'].append(meth_str)
                        read_dict['meC'].append(meC)
                        read_dict['unC'].append(unC)
                        read_dict['cpg_count'].append(cpg_count)
                        read_dict['read_sequence'].append(read_sequence)
                        read_dict['reference_sequence'].append(reference_sequence)
#                         read_object_dict[read.query_name] = read



                    if verbose:
                        print(meth_str)
        if log_dir:
            try:
                sample = file.split('.')[0]
                bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])
                log_file = log_dir+f'bam_metrics_{file.split(".")[3]}.csv'
                if os.path.exists(log_file):
                    all_bam_metrics_df = pd.read_csv(log_file)
                    all_bam_metrics_df = all_bam_metrics_df.merge(bam_metrics_df.reset_index(),on='index')
                    all_bam_metrics_df.to_csv(log_file,index=False)
                else:
                    bam_metrics_df.reset_index().to_csv(log_file,index=False)
            except:
                pass


    if save:
        print(f"Converted {file} to {dest_dir + savename} ")
    else:
        if read_objects:
            return read_encodings,read_dict,read_object_dict
        else:
            return read_encodings,read_dict

In [5]:
# bam_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/taps_dmrs/filtered_bams/"

# filename = "Healthy_X3158.cfTAPS_native_switching_regions2.bam"
# savename = ''
# dest_dir = ''
# read_encodings,read_dict,read_object_dict = taps_bam_to_read_nov_22(filename,savename,bam_dir,dest_dir,
#                              log_dir=log_dir,trimmed=False,save=False,read_objects=True)

In [55]:
filtered_bam_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/taps_dmrs/filtered_bams/"
filtered_reads_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/taps_dmrs/filtered_reads/"

file_list = os.listdir(filtered_bam_dir)

In [60]:
len(file_list)

56

### Wrap into a function

In [41]:
save = False
trimmed = False
verbose=False
include_indels = True

def taps_bam_to_read_indel_aware(file,savename,processed_bam_dir,dest_dir,log_dir=None,trimmed=True,save=True,read_objects=False,include_indels=True,verbose=False):

    """
    New version of TAPS bam2read converter (20/05/2022). 
    
    The previous version was outputting different length
    meth_str and reference_sequence strings. This was due to bad handling of insertions & deletions, where the
    correspondence between reference_sequence and read_sequence is lost. This correspondence is vital as it is 
    used to identify c->T and g->A transitions in each read (OT and OB strand respectively).
    

    This new function properly deals with insertions and deletions, by maintaining correspondence between read_sequence
    reference_sequence and calculated meth_str. 
    
    Outputs:
        If save==True, for each read saves:
            chr
            start
            reference_sequence
            meth_str (should always have same length as reference_sequence)
        If save==False:
            Outputs the above into a read_df, alongside which we can optionally output each read_object for manual inspection (by setting read_objects=True)
    
    """
    sample = file.split('.')[0]
    strand_mutation_mapping = {}
    strand_mutation_mapping[83] = ['g','A']
    strand_mutation_mapping[99] = ['c','T']
    read_object_dict = {}
    strand_cpg_check = {}
    strand_cpg_check[83] = 'Cg'
    strand_cpg_check[99] = 'cG'

    hg38_dir = '/well/ludwig/users/dyp502/read_classifier/hg38.fa'
    genes = Fasta(hg38_dir) 
    chroms = ["chr"+str(i) for i in range(1,23)]
    
    
    if save:
        output = open(dest_dir + savename, 'w')
    
    read_encodings = []
    read_dict = collections.defaultdict(list)
    read_object_dict = {}
    read_count = 0 
    bam_metrics = Counter()
    if (file[-4:]=='.bam'):
        print("converting " + file)
        input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

        for read in input:
            bam_metrics['total_read_count'] +=1
            name = read.reference_name
            if name in chroms:
                bam_metrics['in_chr_1_22'] +=1
                if (read.is_read1) & (read.flag in [99,83]):  # Only interested in read 1 from each read pair
                    bam_metrics['read_1_and_99_83'] +=1
                    methylation_base_change = strand_mutation_mapping[read.flag]
                    reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
                    read_sequence = read.query_sequence # get sequence of read

                    start = str(read.reference_start)
                    end = str(read.reference_end)
                    # Loop through each position in get_aligned_pairs tuple
                    methylation_base_change = strand_mutation_mapping[read.flag]


                    if verbose:
                        print(f'\n\nRead {read_count}:\n')
                        print(name + ": " + start + '-' + end)
                        print(read.flag)
                        print(reference_sequence)
                        print(read_sequence)
                    meth_str = ""

                    if ('I' in read.cigarstring and include_indels) or ('D' in read.cigarstring and include_indels):
                        if verbose:
                            print("Insertion or deletion in read") 
                        bam_metrics['insertion_or_deletion'] +=1
                        read_index,ref_index = 0,0  # insertion/deletion aware read and reference position tracker
                                                    # this tracks the actual position on the read and ref sequence, allowing for deletions and insertions
                        read_aligned_pairs = read.get_aligned_pairs(with_seq=True)
                        for k,(read_pos,ref_pos,ref_base) in enumerate(read_aligned_pairs):
                            insertion=False
                            deletion=False
                            # If there is an indel at this position, there can be no CpG methylation so we skip it
                            if read_pos==None: # if read_pos is none, then deletion in read at this position
                                deletion=True
                                ref_index +=1
                                bam_metrics['deletion'] +=1
                                meth_str += "0" # if deletion, then reference_sequence becomes 1bp longer than read sequence. meth_str must match reference_sequence, so we append 0 to meth_str at this position
                            elif ref_base==None: # if ref_base is none, then insertion in read at this position
                                insertion=True
                                read_index +=1
                                bam_metrics['insertion'] +=1
                                # if insertion, then reference_sequence becomes 1bp shorter than read sequence. meth_str must match reference_sequence, so we don't append to meth_str at insertion position

        #                         if ref_index==len(reference_sequence): # an insertion at the end of the reference_sequence will break the indexing
        #                             bam_metrics['insertion_at_end_of_read'] +=1
        #                             if verbose:
        #                                 print('Insertion at very end of reference sequence, skipping remaining bases on read')
        #                             continue # go back to the top if insertion at end of sequence, ie skip over remaining bases
                            else:
                                if verbose:
                                    print(k,(read_pos,ref_pos,ref_base))
                                    print(f"ref_index: {ref_index}")
                                    print(f"read_index: {read_index}")
                                    print(reference_sequence[ref_index])
                                    print(read_sequence[read_index])

                            #     if not read_pos==None: # if there is no deletion (read_pos is None when deletion)
                                if (reference_sequence[ref_index] == methylation_base_change[0]) & (read_sequence[read_index] == methylation_base_change[1]): # Require either cT or gA
                                    if verbose:
                                        print("Correct change observed, now just need to check CpG context")
                            #         if (ref_base == methylation_base_change[0]) & (read_sequence[read_pos] == methylation_base_change[1]): # Require either cT or gA
                                    context = genes[name][int(start)+ref_index-1:int(start)+ref_index+2] # pyfaidx genes are zero indexed so need to shift -1
                                    if 'cg' in context.seq.lower():
                                        if verbose:
                                            print('context contains CpG so CpG methylation')
                                        meth_str += "1"
                                        bam_metrics['cpg_methylation'] +=1
                                    else:
                                        meth_str += "0"
                                        bam_metrics['non_cpg_methylation'] +=1

                                else:
                                    meth_str += "0"
                                read_index +=1 
                                ref_index +=1 


                    else:
                        bam_metrics['no_insertion_or_deletion'] +=1
                        for k,ref_base in enumerate(reference_sequence):
                            if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
                                bam_metrics['methylation_change'] +=1
                                if verbose:
                                    print("Correct change observed, now just need to check CpG context")

                                if (k==len(reference_sequence)-1) and (read.flag==99): # if methylation on final base of OT, need to check that subsequent base is G
                                    if verbose:
                                        print('Need additional context to check if CpG methylation')
                                    context = genes[name][int(start)+k-1:int(start)+k+2] 
                                    if 'CG' in context:
                                        if verbose:
                                            print('context contains CpG so CpG methylation')
                                        meth_str += "1"
                                        bam_metrics['cpg_methylation'] +=1
                                    else:
                                        if verbose:
                                            print('No CpG in context so non-CpG methylation')
                                        meth_str += "0"
                                        bam_metrics['non_cpg_methylation'] +=1

                                elif (k==0) and (read.flag==83): # if methylation on very first base of OB, need to check that preceding base was C
                                    if verbose:
                                        print('Need additional context to check if CpG methylation')
                                    context = genes[name][int(start)+k-1:int(start)+k+2] 
                                    if 'CG' in context:
                                        if verbose:
                                            print('context contains CpG so CpG methylation')
                                        meth_str += "1"
                                        bam_metrics['cpg_methylation'] +=1

                                    else:
                                        if verbose:
                                            print('No CpG in context so non-CpG methylation')
                                        meth_str += "0"
                                        bam_metrics['non_cpg_methylation'] +=1


                                else:
                                    if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
                                        context = genes[name][int(start)+k-1:int(start)+k+2] 
                                        if verbose:
                                            print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
                                            print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
                                            print(context)
                                        meth_str += "1"
                                        bam_metrics['cpg_methylation'] +=1

                                    else:
                                        meth_str += "0"
                                        if verbose:
                                            print('Non-CpG methylation')
                                        bam_metrics['non_cpg_methylation'] +=1

                            else:
                                meth_str += "0"

                    if len(meth_str) != len(reference_sequence):
                        print('Meth str length does not match reference sequence length so error')
                        break

                    if trimmed:
                        reference_sequence = reference_sequence[5:71]
                        meth_str = meth_str[5:71]
                        read_sequence = read_sequence[5:71]
                    if save:
                        output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
                    else:
                        read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                        read_dict['read_name'].append(read.query_name)
                        read_dict['strand'].append(read.flag)
                        read_dict['chr'].append(read.reference_name)
                        read_dict['start_pos'].append(start)
                        read_dict['ref_sequence'].append(reference_sequence.upper())
                        read_dict['read_sequence'].append(read_sequence.upper())
                        read_dict['meth_str'].append(meth_str)
                        read_object_dict[read.query_name] = read



                    if verbose:
                        print(meth_str)
        if log_dir:
            bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])
            log_file = log_dir+'bam_metrics.csv'
            if os.path.exists(log_file):
                all_bam_metrics_df = pd.read_csv(log_file)
                all_bam_metrics_df[sample] = bam_metrics_df[sample]
                all_bam_metrics_df.to_csv(log_file)
            else:
                bam_metrics_df.to_csv(log_file)


    if save:
        print(f"Converted {file} to {dest_dir + savename} ")
    else:
        if read_objects:
            return read_encodings,read_dict,read_object_dict
        else:
            return read_encodings,read_dict



### Test function

In [22]:
processed_bam_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/filtered_bams/dismir_10x_switching_regions/split_2/"
healthy_cfdna = list(sample_metadata_df.loc[(sample_metadata_df['status'].isin(['healthy_cfdna']))]['samples'])
# sample = 'CTR134'
# sample = 'N2L_trimmed_CD'
sample = 'Healthy_X2875'
file = [u for u in os.listdir(processed_bam_dir) if sample in u][0]
# input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')
log_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/logs/"
savename=''
dest_dir=''




# read_encodings,read_dict = taps_bam_to_read_indel_aware(file,savename,processed_bam_dir,dest_dir,log_dir=log_dir,save=False,trimmed=False)



In [27]:
len(read.seq)

116

In [39]:
bam_metrics_df

,Healthy_X2875
total_read_count,2421
in_chr_1_22,2421
read_1_and_99_83,1001
no_insertion_or_deletion,989
methylation_change,3893
cpg_methylation,3792
insertion_or_deletion,12
deletion,8
non_cpg_methylation,148
insertion,17


### Further testing (Oct 2022)

- Testing on CancerDetector and TAPS samples, 

In [42]:
processed_bam_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/cftaps_native_switching_regions/filtered_bams/"
file = "HCC2.cfTAPS_native_switching_regions.hyper_hypo.threshold_0.7.split_1.bam"
# healthy_cfdna = list(sample_metadata_df.loc[(sample_metadata_df['status'].isin(['healthy_cfdna']))]['samples'])
# sample = 'CTR134'
# sample = 'N2L_trimmed_CD'
# sample = 'Healthy_X2875'
# file = [u for u in os.listdir(processed_bam_dir) if sample in u][0]
# input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')


log_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/logs/"
savename=''
dest_dir=''

# input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')
# for read in input:
#     pass
read_encodings,read_dict = taps_bam_to_read_indel_aware(file,savename,processed_bam_dir,dest_dir,log_dir=log_dir,save=False,trimmed=False,verbose=True)



converting HCC2.cfTAPS_native_switching_regions.hyper_hypo.threshold_0.7.split_1.bam


In [47]:
if (file[-4:]=='.bam'):
    print("converting " + file)
    input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

    for read in input:
        bam_metrics['total_read_count'] +=1
        name = read.reference_name
        if name in chroms:
#             print('name in chroms')
            bam_metrics['in_chr_1_22'] +=1
            if (read.is_read1) & (read.flag in [99,83]):  # Only interested in read 1 from each read pair
                print("read is read1")
                bam_metrics['read_1_and_99_83'] +=1
                methylation_base_change = strand_mutation_mapping[read.flag]
                reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
                read_sequence = read.query_sequence # get sequence of read

                start = str(read.reference_start)
                end = str(read.reference_end)
                # Loop through each position in get_aligned_pairs tuple
                methylation_base_change = strand_mutation_mapping[read.flag]


                if verbose:
                    print(f'\n\nRead {read_count}:\n')
                    print(name + ": " + start + '-' + end)
                    print(read.flag)
                    print(reference_sequence)
                    print(read_sequence)
                meth_str = ""


converting HCC2.cfTAPS_native_switching_regions.hyper_hypo.threshold_0.7.split_1.bam


KeyboardInterrupt: 

In [51]:
experiments_dir = "/well/ludwig/users/dyp502/read_classifier/dismir_experiments/"
with open(experiments_dir + 'sample_metadata_dict.pickle', 'rb') as handle:
    sample_metadata_dict = pickle.load(handle)

In [53]:
SAMPLES = ['CTR148', 'Healthy_X3156', 'HOT222', 'Healthy_BB2850', 'HOT151T', 'CTR147', 'CTR86', 'CTR113', 'HOT238T', 'CTR154', 'CTR152', 'HOT208', 'Healthy_BB2853', 'HOT222T', 'HOT198', 'HCC_NHCCDAHA038', 'HOT192', 'HCC_NHCCLETA034', 'HOT159', 'HCC_NHCCKAMI035', 'N1L', 'CTR134', 'Healthy_X2875', 'Healthy_IBD747', 'HCC_NHCCFRWA050', 'HCC_NHCCDAPA022', 'HOT207T', 'HCC_NHCCLIMC055', 'HCC_NHCCJATH017', 'HOT197', 'HOT197T', 'Healthy_X3165', 'HCC_NHCCDACU015', 'HOT172T', 'Healthy_BB2833', 'HOT207', 'Healthy_X3162', 'Healthy_X2654', 'HCC_NHCCFRLU064', 'HCC_NHCCJAGO048', 'HOT240', 'LIC2L', 'Healthy_TP277', 'Healthy_X2882', 'Healthy_IBD3087', 'Healthy_BB2844', 'HOT236', 'HCC_NHCCRAWI047', 'HOT172', 'CTR151', 'Healthy_X3421', 'N3L', 'Healthy_BB2783', 'Healthy_X2012', 'CTR101', 'HOT205', 'HOT170', 'LIC1L', 'HCC_NHCCIAPA010', 'HOT233', 'Healthy_BB2802', 'Healthy_BB2316', 'CTR110', 'HOT204', 'CTR103', 'CTR153', 'HCC_NHCCIABR006', 'HCC2', 'LIC4L', 'HOT164', 'N2L', 'CTR127', 'HCC_NHCCHEDO065', 'Healthy_IBD3642', 'Healthy_X1083', 'HOT240T', 'HCC1', 'HCC_NHCCERTR058', 'CTR128', 'Healthy_X3158', 'HOT227', 'CTR111', 'HOT265', 'HOT229T', 'Healthy_NHCCJORO037', 'HOT167', 'CTR106', 'HOT273', 'HOT151', 'CTR126', 'HCC_NHCCDARI033', 'HOT236T', 'HOT229', 'HCC_NHCCDEHA060', 'CTR114', 'Healthy_X3161', 'HCC_NHCCMIWH056', 'HCC_NHCCJOHO063', 'HOT227T', 'CTR104', 'Healthy_X4156', 'HOT162', 'Healthy_GI3234', 'HOT243', 'CTR129', 'CTR98', 'HOT224', 'CTR97', 'Healthy_X2881', 'CTR132', 'CTR85', 'CTR117', 'CTR150', 'HOT170T', 'HOT215T', 'LIC3L', 'Healthy_GI4154', 'Healthy_X3368', 'CTR149', 'HCC_NHCCANPA054', 'Healthy_X3533', 'CTR84', 'Healthy_BB2877', 'HCC_NHCCGEMU051', 'Healthy_X2838', 'CTR118', 'HOT156', 'HOT238', 'HOT215', 'CTR108', 'CTR131', 'HCC_NHCCLEHO053', 'N4L', 'HOT205T', 'CTR107']


In [60]:
sample = "LIC2L_trimmed_CD"


True

In [58]:
sample_metadata_dict.keys()

{'N1L_trimmed_CD': ['healthy_cfdna', 'cancer_detector'],
 'N2L_trimmed_CD': ['healthy_cfdna', 'cancer_detector'],
 'N3L_trimmed_CD': ['healthy_cfdna', 'cancer_detector'],
 'N4L_trimmed_CD': ['healthy_cfdna', 'cancer_detector'],
 'LIC1L_trimmed_CD': ['hcc_cfdna', 'cancer_detector'],
 'LIC2L_trimmed_CD': ['hcc_cfdna', 'cancer_detector'],
 'LIC3L_trimmed_CD': ['hcc_cfdna', 'cancer_detector'],
 'LIC4L_trimmed_CD': ['hcc_cfdna', 'cancer_detector'],
 'HCC1_trimmed_CD': ['hcc_tumour', 'cancer_detector'],
 'HCC2_trimmed_CD': ['hcc_tumour', 'cancer_detector'],
 'Healthy_IBD747': ['healthy_cfdna', 'TAPS'],
 'Healthy_X2875': ['healthy_cfdna', 'TAPS'],
 'Healthy_X2838': ['healthy_cfdna', 'TAPS'],
 'Healthy_X3156': ['healthy_cfdna', 'TAPS'],
 'Healthy_X2882': ['healthy_cfdna', 'TAPS'],
 'Healthy_BB2853': ['healthy_cfdna', 'TAPS'],
 'Healthy_X2881': ['healthy_cfdna', 'TAPS'],
 'Healthy_BB2850': ['healthy_cfdna', 'TAPS'],
 'Healthy_X3162': ['healthy_cfdna', 'TAPS'],
 'Healthy_BB2877': ['healthy_cfdna

In [57]:
rough_sample_match
sample_metadata_dict['N1L'][1]

KeyError: 'N1L'

#### What should we do when an insertion means read length (and hence meth_str) is a different length from reference length?

Options:

1. Ignore all insertions and deletions

2. Account for position somehow so that methylation position reflects correct position in read database



In [53]:
# read.get_aligned_pairs(with_seq=True)

In [21]:
read.cigarstring

'13M1D100M'

In [22]:
len(meth_str)

114

In [25]:
read_index,ref_index = 0,0

read_aligned_pairs = read.get_aligned_pairs(with_seq=True)
for k,(read_pos,ref_pos,ref_base) in enumerate(read_aligned_pairs):
    print(k,(read_pos,ref_pos,ref_base))
    print(f"ref_index: {ref_index}")
    print(f"read_index: {read_index}")
    print(reference_sequence[ref_index])
    print(read_sequence[read_index])
#     if not read_pos==None: # if there is no deletion (read_pos is None when deletion)
    if (reference_sequence[ref_index] == methylation_base_change[0]) & (read_sequence[read_index] == methylation_base_change[1]): # Require either cT or gA
#         if (ref_base == methylation_base_change[0]) & (read_sequence[read_pos] == methylation_base_change[1]): # Require either cT or gA
        context = genes[name][ref_pos-1:ref_pos+2] # pyfaidx genes are zero indexed so need to shift -1
        if 'cg' in context.seq.lower():
#                             if (read_aligned_pairs[k-1][2] == 'C') or (read_aligned_pairs[k+1][2] == 'G'): # Make sure it's CpG context
            methylation +=1
            meth_str += "1"
        else:
            meth_str += "0"
    else:
        meth_str += "0"
    length_count += 1
    
    
    if not read_pos==None:
        read_index +=1
    if not ref_base==None:
        ref_index +=1 
    
    print(f"iteration {k}\n\n")

0 (0, 1178392, 'C')
ref_index: 0
read_index: 0
C
C
iteration 0


1 (1, 1178393, 'T')
ref_index: 1
read_index: 1
T
T
iteration 1


2 (2, 1178394, 'A')
ref_index: 2
read_index: 2
A
A
iteration 2


3 (3, 1178395, 'C')
ref_index: 3
read_index: 3
C
C
iteration 3


4 (4, 1178396, 'T')
ref_index: 4
read_index: 4
T
T
iteration 4


5 (5, 1178397, 'A')
ref_index: 5
read_index: 5
A
A
iteration 5


6 (6, 1178398, 'A')
ref_index: 6
read_index: 6
A
A
iteration 6


7 (7, 1178399, 'A')
ref_index: 7
read_index: 7
A
A
iteration 7


8 (8, 1178400, 'A')
ref_index: 8
read_index: 8
A
A
iteration 8


9 (9, 1178401, 'A')
ref_index: 9
read_index: 9
A
A
iteration 9


10 (10, 1178402, 'T')
ref_index: 10
read_index: 10
T
T
iteration 10


11 (11, 1178403, 'A')
ref_index: 11
read_index: 11
A
A
iteration 11


12 (12, 1178404, 'C')
ref_index: 12
read_index: 12
C
C
iteration 12


13 (None, 1178405, 'A')
ref_index: 13
read_index: 13
A
A
iteration 13


14 (13, 1178406, 'A')
ref_index: 14
read_index: 13
A
A
iteration 14


In [24]:
ref_index

149

In [25]:
reference_sequence[ref_index]

IndexError: string index out of range

In [1]:
read_encodings = []
read_dict = collections.defaultdict(list)
read_object_dict = {}
save = False
trimmed = False
read_count = 0 
bam_metrics = Counter()
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

for read in input:
    bam_metrics['total_read_count'] +=1
    if (read.is_read1) & (read.flag in [99,83]):
        bam_metrics['read_1_and_99_83'] +=1
        if ('I' not in read.cigarstring) & ('D' not in read.cigarstring):
            bam_metrics['no_mutation_deletion'] +=1
#         if 'M' in read.cigarstring:
     # Only interested in read 1 from each read pair
            methylation_base_change = strand_mutation_mapping[read.flag]
            reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
            read_sequence = read.query_sequence # get sequence of read
            name = read.reference_name
            if name in chroms:
                bam_metrics['in_chr_1_22'] +=1
                start = str(read.reference_start)
                end = str(read.reference_end)
                # Loop through each position in get_aligned_pairs tuple
                # If c-->T (or g-->A for OB strand), then add "1" to meth_list, otherwise add "0"
                methylation_base_change = strand_mutation_mapping[read.flag]
                meth_str = ""
                for k,ref_base in enumerate(reference_sequence):
                    if (ref_base==methylation_base_change[0]) & (read_sequence[k]==methylation_base_change[1]):
                        bam_metrics['methylation_change'] +=1
                        print("Correct change observed, now just need to check CpG context")
                        
                        if (k==len(reference_sequence)-1) and (read.flag==99): # if methylation on final base of OT, need to check that subsequent base is G
                            print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1
                            else:
                                print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1

                        elif (k==0) and (read.flag==83): # if methylation on very first base of OB, need to check that preceding base was C
                            print('Need additional context to check if CpG methylation')
                            context = genes[name][int(start)+k-1:int(start)+k+2] 
                            if 'CG' in context:
                                print('context contains CpG so CpG methylation')
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                print('No CpG in context so non-CpG methylation')
                                meth_str += "0"
                                bam_metrics['non_cpg_methylation'] +=1


                        else:
                            if strand_cpg_check[read.flag] in reference_sequence[k-1:k+2]:
                                print(f"{methylation_base_change[0]} to {methylation_base_change[1]} transition at position {k}, so methylation")
                                print(f"{strand_cpg_check[read.flag]} is in {reference_sequence[k-1:k+2]}, so CpG site")
                                context = genes[name][int(start)+k-1:int(start)+k+2] 
                                print(context)
                                meth_str += "1"
                                bam_metrics['cpg_methylation'] +=1

                            else:
                                meth_str += "0"
                                print('Non-CpG methylation')
                                bam_metrics['non_cpg_methylation'] +=1

                    else:
                        meth_str += "0"
                        
            if len(meth_str) != len(reference_sequence):
                print('Meth str does not match sequence so error')
                break
                
            if trimmed:
                reference_sequence = reference_sequence[5:71]
                meth_str = meth_str[5:71]
                read_sequence = read_sequence[5:71]
            if save:
                output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
            else:
                read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                read_dict['read_name'].append(read.query_name)
                read_dict['strand'].append(read.flag)
                read_dict['chr'].append(read.reference_name)
                read_dict['start_pos'].append(start)
                read_dict['ref_sequence'].append(reference_sequence.upper())
                read_dict['read_sequence'].append(read_sequence.upper())
                read_dict['meth_str'].append(meth_str)
                read_object_dict[read.query_name] = read


            print(f'\n\nRead {read_count}:\n')
            print(name + ": " + start + '-' + end)
            print(read.flag)
            print(reference_sequence)
            print(read_sequence)
            print(meth_str)
            read_count +=1
            
            
            if read_count > 100000:
                break
                
bam_metrics_df = pd.DataFrame.from_dict(dict(bam_metrics),orient='index',columns=[sample])


In [68]:
for k,row in read_df.iterrows():
    if len(row['meth_str']) != len(row['ref_sequence']):
        print(k,row)

In [69]:
bam_metrics_df

,Healthy_X2875
read_1_and_99_83,102103
no_mutation_deletion,100001
in_chr_1_22,100001
methylation_change,319166
cpg_methylation,306730
non_cpg_methylation,12436


In [120]:
read_encodings = []
read_dict = collections.defaultdict(list)
read_object_dict = {}
save = False
trimmed = False
read_count = 0
input = pysam.AlignmentFile(processed_bam_dir + file, 'rb')

for read in input:
    if (read.is_read1) & (read.flag in [99,83]):
        if ('I' not in read.cigarstring) & ('D' not in read.cigarstring):
#         if 'M' in read.cigarstring:
     # Only interested in read 1 from each read pair
            methylation_base_change = strand_mutation_mapping[read.flag]
            reference_sequence = read.get_reference_sequence() # get sequence of reference with mismatches in lower case 
            read_sequence = read.query_sequence # get sequence of read
            name = read.reference_name
            if name in chroms:
                start = str(read.reference_start)
                end = str(read.reference_end)
                # read_length = len(read_sequence)

                # Loop through each position in get_aligned_pairs tuple
                # If c-->T (or g-->A for OB strand), then add "1" to meth_list, otherwise add "0"
                meth_str = ""
                methylation = 0 # track whether each read is methylated or not 
                read_aligned_pairs = read.get_aligned_pairs(with_seq=True)
                if len(read_aligned_pairs) != len(reference_sequence):
                    print('read aligned pairs diff length than reference sequence')
                length_count = 0
                ref_sequence_from_aligned_pairs = ""
                for k,(read_pos,ref_pos,ref_base) in enumerate(read_aligned_pairs):
                    if not read_pos==None: # if there is no deletion (read_pos is None when deletion)
    #                     ref_sequence_from_aligned_pairs += ref_base
                        if (ref_base == methylation_base_change[0]) & (read_sequence[read_pos] == methylation_base_change[1]): # Require either cT or gA
                            context = genes[name][ref_pos-1:ref_pos+2] # pyfaidx genes are zero indexed so need to shift -1
                            if 'cg' in context.seq.lower():
        #                             if (read_aligned_pairs[k-1][2] == 'C') or (read_aligned_pairs[k+1][2] == 'G'): # Make sure it's CpG context
                                methylation +=1
                                meth_str += "1"
                            else:
                                meth_str += "0"
                        else:
                            meth_str += "0"
                    read_count +=1

                if len(read_sequence[5:71]) != len(meth_str[5:71]):
                    print("broken at " + read.query_name)
                    break
                else:
                    if trimmed:
                        reference_sequence = reference_sequence[5:71]
                        meth_str = meth_str[5:71]
                        read_sequence = read_sequence[5:71]
                    if save:
                            # output.write(name+'\t'+start+'\t'+read_sequence+'\t'+meth_str+'\n')
                            output.write(name+'\t'+start+'\t'+reference_sequence.upper()+'\t'+meth_str+'\n')
                    else:
                            read_encodings.append(name+'\t'+start+'\t'+read_sequence.upper()+'\t'+meth_str+'\n')
                            read_count +=1
                            read_dict['read_name'].append(read.query_name)
                            read_dict['strand'].append(read.flag)
                            read_dict['chr'].append(read.reference_name)
                            read_dict['start_pos'].append(start)
                            read_dict['ref_sequence'].append(reference_sequence.upper())
                            read_dict['read_sequence'].append(read_sequence.upper())
                            read_dict['meth_str'].append(meth_str)
                            read_object_dict[read.query_name] = read

print(read.flag)
print(methylation_base_change)
print(reference_sequence)
print(read_sequence)
print(name)
print(start)
print(end)
# print(read_aligned_pairs)

read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read aligned pairs diff length than reference sequence
read align

KeyboardInterrupt: 

In [107]:
read_df = pd.DataFrame.from_dict(read_dict)

In [108]:
read_df.head()

,read_name,strand,chr,start_pos,ref_sequence,read_sequence,meth_str
0,A00711:132:HYJY2DSXX:3:1455:22236:13041,99,chr1,1178369,ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAG...,ACCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAG...,0000000000000000000000000000000000000000000000...
1,A00711:132:HYJY2DSXX:2:1633:4372:11600,83,chr1,1178370,CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGC...,CCAACATGGAGAAAACCCATCTCTACTAAAAATACAAAAAGTTAGC...,0000000000000000000000000000000000000000000000...
2,A00711:132:HYJY2DSXX:2:1225:16975:8437,99,chr1,1178386,CCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGG...,CCATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGG...,0000000000000000000000000000000000000000000000...
3,A00711:132:HYJY2DSXX:4:1438:9272:16172,83,chr1,1178387,CATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGT...,CATCTCTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGT...,0000000000000000000000000000000000000000000000...
4,A00551:125:H2YGFDSXY:4:2257:13838:29528,83,chr1,1178392,CTACTAAAAATACAAAAAGTTAGCTGGGCATGGTGGTGGGTGCCTG...,CTACTAAAAATACAAACAGTTAGCTGGGCATGGTGGTGGGTGCCTG...,0000000000000000000000000000000000000000000000...


In [109]:
for k,row in read_df.iterrows():
    if len(row['ref_sequence'])!= len(row['meth_str']):
        print(k,row['read_name'])

45 A00711:132:HYJY2DSXX:1:1175:15935:12336
58 A00711:132:HYJY2DSXX:4:1371:17716:8531
60 A00711:163:H5F5HDSXY:3:1272:28782:5995
62 A00711:163:H5F5HDSXY:4:1365:22670:2268
64 A00551:125:H2YGFDSXY:3:1515:10212:5368
65 A00711:132:HYJY2DSXX:2:1267:22345:9846
68 A00711:163:H5F5HDSXY:3:1138:9182:36714
69 A00711:132:HYJY2DSXX:2:1655:17472:6355
72 A00551:125:H2YGFDSXY:1:1530:28022:8656
80 A00711:163:H5F5HDSXY:3:1419:2772:14559
81 A00711:132:HYJY2DSXX:3:1205:14055:6292
88 A00711:132:HYJY2DSXX:2:2505:21160:13651
114 A00711:163:H5F5HDSXY:1:2230:16749:5353
116 A00711:132:HYJY2DSXX:1:1255:24749:33113
123 A00551:125:H2YGFDSXY:2:1124:14516:21026
158 A00551:125:H2YGFDSXY:2:1202:3866:34898
159 A00711:163:H5F5HDSXY:3:2411:26784:2942
250 A00711:132:HYJY2DSXX:4:1263:10276:9111
260 A00711:132:HYJY2DSXX:1:1533:23981:11898
288 A00711:163:H5F5HDSXY:2:2104:31711:4899
297 A00711:132:HYJY2DSXX:2:1445:4689:3599
300 A00711:132:HYJY2DSXX:3:1237:29261:28839
305 A00551:125:H2YGFDSXY:3:1613:21151:13792
306 A00711:163:H5

8185 A00711:132:HYJY2DSXX:1:1514:27832:20635
8228 A00551:125:H2YGFDSXY:4:2520:16514:21324
8303 A00711:132:HYJY2DSXX:4:1254:5249:25582
8365 A00551:125:H2YGFDSXY:2:1625:10619:2566
8393 A00711:163:H5F5HDSXY:4:2121:13033:4022
8437 A00551:125:H2YGFDSXY:3:2316:26223:30937
8493 A00551:125:H2YGFDSXY:4:2357:30327:4069
8495 A00551:125:H2YGFDSXY:1:1166:27887:10551
8623 A00551:125:H2YGFDSXY:4:1644:27787:30921
8630 A00711:132:HYJY2DSXX:3:1247:16260:8015
8770 A00711:132:HYJY2DSXX:3:2463:26313:23453
8774 A00551:125:H2YGFDSXY:4:1110:24921:28087
8775 A00711:163:H5F5HDSXY:1:2557:25201:35023
8776 A00711:163:H5F5HDSXY:2:1638:27118:35618
8792 A00711:163:H5F5HDSXY:1:1122:8142:7044
8808 A00551:125:H2YGFDSXY:2:1262:6470:16485
8818 A00711:132:HYJY2DSXX:2:2160:9299:26647
8825 A00711:132:HYJY2DSXX:4:1233:19886:34522
8835 A00551:125:H2YGFDSXY:3:1565:2871:8500
8836 A00711:132:HYJY2DSXX:4:1325:24306:2472
8844 A00711:132:HYJY2DSXX:3:1431:20401:5541
8845 A00711:132:HYJY2DSXX:3:2612:22426:24862
8846 A00711:163:H5F5HDS

14283 A00551:125:H2YGFDSXY:2:1444:15329:37027
14290 A00711:163:H5F5HDSXY:4:1655:15845:28463
14362 A00551:125:H2YGFDSXY:2:1166:10285:34021
14377 A00711:163:H5F5HDSXY:3:1468:29188:7670
14382 A00711:163:H5F5HDSXY:2:2521:18069:36104
14383 A00551:125:H2YGFDSXY:3:1623:16839:33285
14384 A00551:125:H2YGFDSXY:2:2170:26142:14137
14385 A00711:132:HYJY2DSXX:1:2533:11143:18411
14386 A00711:132:HYJY2DSXX:2:1221:5122:23954
14389 A00711:132:HYJY2DSXX:1:1451:21947:29512
14490 A00711:132:HYJY2DSXX:4:2146:15709:17300
14596 A00711:132:HYJY2DSXX:4:2647:5701:26897
14766 A00711:163:H5F5HDSXY:3:2229:30382:15749
14788 A00711:163:H5F5HDSXY:3:2658:14606:33927
14803 A00711:132:HYJY2DSXX:2:1578:30779:30467
14806 A00551:125:H2YGFDSXY:2:1431:24343:29997
14810 A00711:163:H5F5HDSXY:3:1603:29279:15530
14862 A00711:163:H5F5HDSXY:3:1633:32542:8625
14872 A00711:132:HYJY2DSXX:1:1273:21169:10504
14952 A00711:132:HYJY2DSXX:4:1206:10095:4257
14953 A00711:163:H5F5HDSXY:2:2563:30382:19633
14974 A00551:125:H2YGFDSXY:2:1255:25952

22577 A00711:163:H5F5HDSXY:1:2608:26666:29105
22604 A00711:132:HYJY2DSXX:1:1117:18873:24158
22624 A00551:125:H2YGFDSXY:2:2274:12472:36151
22665 A00711:132:HYJY2DSXX:4:1559:16224:19570
22674 A00551:125:H2YGFDSXY:3:2149:1127:15248
22678 A00711:163:H5F5HDSXY:3:2163:31855:17675
22719 A00551:125:H2YGFDSXY:2:1408:10791:19961
22826 A00551:125:H2YGFDSXY:3:2363:19786:30655
22992 A00711:163:H5F5HDSXY:4:2413:30038:7294
23170 A00711:132:HYJY2DSXX:3:2636:16405:21136
23173 A00551:125:H2YGFDSXY:1:2410:19199:1861
23204 A00711:132:HYJY2DSXX:3:2460:4689:15749
23206 A00711:163:H5F5HDSXY:2:1558:20871:34569
23246 A00711:132:HYJY2DSXX:3:1415:29396:30859
23274 A00711:132:HYJY2DSXX:4:1347:5321:14716
23279 A00551:125:H2YGFDSXY:1:2640:10203:22482
23330 A00711:163:H5F5HDSXY:3:2361:9236:15483
23379 A00711:132:HYJY2DSXX:2:1115:11153:28197
23385 A00711:132:HYJY2DSXX:3:1518:8314:28761
23467 A00551:125:H2YGFDSXY:1:1243:5086:20478
23554 A00551:125:H2YGFDSXY:4:1254:29894:27586
23661 A00711:132:HYJY2DSXX:3:2625:10945:17

30668 A00711:132:HYJY2DSXX:4:2446:3025:14685
30687 A00551:125:H2YGFDSXY:2:2274:5791:28275
30711 A00711:163:H5F5HDSXY:2:2262:32922:16047
30719 A00711:132:HYJY2DSXX:3:2563:20826:8907
30729 A00551:125:H2YGFDSXY:1:1366:12057:31579
30777 A00551:125:H2YGFDSXY:2:1435:7744:9173
30779 A00551:125:H2YGFDSXY:4:2533:26992:6746
30780 A00711:132:HYJY2DSXX:4:1349:5773:10614
30844 A00551:125:H2YGFDSXY:1:1108:30355:12978
30898 A00711:132:HYJY2DSXX:1:1459:29930:8171
30912 A00551:125:H2YGFDSXY:3:2262:3568:14904
30915 A00551:125:H2YGFDSXY:2:1425:11478:7874
30916 A00551:125:H2YGFDSXY:2:1407:22037:5995
30917 A00711:132:HYJY2DSXX:3:2667:12319:15248
30976 A00711:132:HYJY2DSXX:4:1212:19397:26287
30988 A00711:132:HYJY2DSXX:4:2276:24614:17440
31176 A00551:125:H2YGFDSXY:4:2201:7184:27899
31183 A00711:163:H5F5HDSXY:2:2277:18719:23985
31186 A00711:132:HYJY2DSXX:2:1304:22670:5400
31189 A00551:125:H2YGFDSXY:4:2457:28384:7905
31210 A00711:132:HYJY2DSXX:4:2232:19153:28025
31217 A00711:163:H5F5HDSXY:2:1274:7410:22561
312

...

106376 A00551:125:H2YGFDSXY:2:1245:17047:27226
106377 A00711:132:HYJY2DSXX:2:1245:29252:36370
106379 A00711:163:H5F5HDSXY:4:1550:15582:14262
106390 A00711:163:H5F5HDSXY:3:2627:13458:3035
106433 A00711:132:HYJY2DSXX:4:1349:29658:11835
106435 A00551:125:H2YGFDSXY:4:1218:18394:10958
106546 A00551:125:H2YGFDSXY:1:1317:6162:16110
106547 A00551:125:H2YGFDSXY:2:2659:29839:5040
106603 A00551:125:H2YGFDSXY:2:1421:5981:11287
106611 A00711:163:H5F5HDSXY:1:2534:1099:34804
106712 A00711:163:H5F5HDSXY:3:1634:9344:20024
106728 A00711:163:H5F5HDSXY:3:2532:4020:32690
106734 A00711:132:HYJY2DSXX:4:1328:14687:26240
106766 A00711:132:HYJY2DSXX:3:1256:28772:16125
106822 A00711:163:H5F5HDSXY:3:2359:10357:24846
106825 A00711:132:HYJY2DSXX:1:1216:18945:26005
106868 A00711:132:HYJY2DSXX:2:1658:12427:12023
106869 A00551:125:H2YGFDSXY:3:1536:28953:7795
106900 A00711:163:H5F5HDSXY:4:1359:17445:11130
107171 A00711:132:HYJY2DSXX:1:2370:1045:29356
107326 A00711:132:HYJY2DSXX:2:2372:2049:35039
107383 A00711:132:HYJY2

114355 A00711:132:HYJY2DSXX:1:1169:9986:10175
114396 A00551:125:H2YGFDSXY:1:2525:19271:14638
114448 A00711:132:HYJY2DSXX:4:1654:27724:16188
114450 A00711:163:H5F5HDSXY:4:2263:8260:16861
114464 A00711:163:H5F5HDSXY:2:2636:19072:31955
114539 A00711:163:H5F5HDSXY:4:1537:23683:29763
114701 A00711:132:HYJY2DSXX:1:1404:13548:17190
114804 A00711:163:H5F5HDSXY:1:2407:15040:31109
114828 A00711:132:HYJY2DSXX:1:1546:7835:32784
114829 A00551:125:H2YGFDSXY:4:2648:19036:12540
114830 A00551:125:H2YGFDSXY:2:1614:16224:8484
114862 A00551:125:H2YGFDSXY:2:1473:14208:15389
114869 A00711:132:HYJY2DSXX:3:1318:23068:10003
114873 A00711:132:HYJY2DSXX:3:2560:3839:27398
114880 A00711:132:HYJY2DSXX:3:2557:19434:29387
114883 A00711:163:H5F5HDSXY:1:2206:18484:13025
114884 A00551:125:H2YGFDSXY:1:1541:10972:30076
114885 A00551:125:H2YGFDSXY:1:1662:30644:1986
114886 A00551:125:H2YGFDSXY:2:1372:2682:19132
114887 A00551:125:H2YGFDSXY:3:2347:21359:14559
114888 A00711:163:H5F5HDSXY:1:2174:29975:24878
114889 A00711:163:H5

119578 A00711:163:H5F5HDSXY:1:1425:29794:24283
119627 A00711:132:HYJY2DSXX:2:1433:20980:2503
119630 A00551:125:H2YGFDSXY:4:1138:9815:8844
119658 A00711:132:HYJY2DSXX:2:1345:19470:11788
119675 A00551:125:H2YGFDSXY:1:1656:24713:17832
119676 A00551:125:H2YGFDSXY:2:1214:9516:10676
119677 A00711:132:HYJY2DSXX:2:2342:11017:7795
119767 A00711:163:H5F5HDSXY:3:2360:10303:30389
119973 A00711:163:H5F5HDSXY:3:2263:9543:2613
119985 A00711:163:H5F5HDSXY:1:1322:12246:34068
120036 A00711:163:H5F5HDSXY:1:1535:21694:2738
120043 A00551:125:H2YGFDSXY:1:1264:12961:33113
120055 A00711:163:H5F5HDSXY:4:2663:7780:12367
120132 A00711:132:HYJY2DSXX:2:1655:16324:29700
120133 A00711:163:H5F5HDSXY:3:2354:15682:24486
120134 A00711:132:HYJY2DSXX:3:1623:2772:7482
120135 A00551:125:H2YGFDSXY:3:2328:29324:11913
120141 A00711:163:H5F5HDSXY:1:2522:27986:7748
120147 A00711:163:H5F5HDSXY:1:2121:21160:30342
120150 A00711:132:HYJY2DSXX:2:1625:14507:28588
120152 A00711:163:H5F5HDSXY:3:1251:18358:11490
120287 A00711:132:HYJY2DS

128773 A00711:132:HYJY2DSXX:2:2522:30047:7185
128774 A00711:132:HYJY2DSXX:3:2626:17897:7404
128782 A00711:132:HYJY2DSXX:2:1215:7401:21292
128990 A00711:163:H5F5HDSXY:3:1120:29423:28275
129018 A00551:125:H2YGFDSXY:4:2430:25834:9846
129035 A00711:163:H5F5HDSXY:3:2429:10493:30248
129049 A00711:132:HYJY2DSXX:4:1619:16812:23625
129058 A00711:132:HYJY2DSXX:1:2402:16306:26944
129070 A00711:163:H5F5HDSXY:3:1558:5584:15295
129078 A00711:163:H5F5HDSXY:1:2127:30716:2675
129113 A00551:125:H2YGFDSXY:4:2425:7301:14794
129119 A00551:125:H2YGFDSXY:3:1652:13612:36746
129120 A00551:125:H2YGFDSXY:4:2309:13123:1579
129210 A00711:132:HYJY2DSXX:2:1555:14000:10551
129255 A00711:163:H5F5HDSXY:3:2125:26856:15875
129256 A00711:132:HYJY2DSXX:3:1153:26494:1313
129257 A00711:163:H5F5HDSXY:2:1225:10312:13150
129258 A00551:125:H2YGFDSXY:1:1348:2456:33646
129259 A00711:132:HYJY2DSXX:3:1306:1497:24940
129260 A00711:163:H5F5HDSXY:4:1143:9625:31626
129262 A00711:132:HYJY2DSXX:3:2210:19081:35008
129281 A00711:163:H5F5HDS

134329 A00711:163:H5F5HDSXY:3:2659:17038:22482
134357 A00711:163:H5F5HDSXY:3:2324:29848:2550
134394 A00711:132:HYJY2DSXX:1:2123:21667:16282
134400 A00711:132:HYJY2DSXX:4:2140:5186:35869
134411 A00711:132:HYJY2DSXX:2:2541:5412:33317
134415 A00551:125:H2YGFDSXY:3:2347:22544:22561
134417 A00551:125:H2YGFDSXY:1:2573:22715:6198
134418 A00711:163:H5F5HDSXY:4:1307:18069:21543
134419 A00551:125:H2YGFDSXY:1:1467:8296:17018
134427 A00711:132:HYJY2DSXX:1:1435:30490:9048
134437 A00711:163:H5F5HDSXY:2:2227:15148:18302
134438 A00711:163:H5F5HDSXY:3:1554:32054:7560
134439 A00711:132:HYJY2DSXX:1:1478:24343:11835
134440 A00711:132:HYJY2DSXX:3:1127:27796:11177
134441 A00551:125:H2YGFDSXY:1:1676:27190:16830
134442 A00711:132:HYJY2DSXX:3:2211:4544:22889
134443 A00711:132:HYJY2DSXX:2:1619:29848:22999
134444 A00551:125:H2YGFDSXY:1:2359:27977:14152
134445 A00551:125:H2YGFDSXY:3:2226:28094:32675
134446 A00551:125:H2YGFDSXY:4:2475:20021:5259
134447 A00711:163:H5F5HDSXY:3:2258:3115:7044
134448 A00711:163:H5F5HD

In [132]:
read = read_object_dict['A00711:132:HYJY2DSXX:1:1175:15935:12336']
len(read.get_aligned_pairs(with_seq=True))

148

In [134]:
read.cigarstring

'144M4S'

In [133]:
read.get_aligned_pairs(with_seq=True)

[(0, 1218885, 'C'),
 (1, 1218886, 'C'),
 (2, 1218887, 'T'),
 (3, 1218888, 'G'),
 (4, 1218889, 'G'),
 (5, 1218890, 'T'),
 (6, 1218891, 'A'),
 (7, 1218892, 'C'),
 (8, 1218893, 'C'),
 (9, 1218894, 'A'),
 (10, 1218895, 'G'),
 (11, 1218896, 'c'),
 (12, 1218897, 'G'),
 (13, 1218898, 'G'),
 (14, 1218899, 'T'),
 (15, 1218900, 'C'),
 (16, 1218901, 'C'),
 (17, 1218902, 'T'),
 (18, 1218903, 'T'),
 (19, 1218904, 'C'),
 (20, 1218905, 'A'),
 (21, 1218906, 'G'),
 (22, 1218907, 'G'),
 (23, 1218908, 'T'),
 (24, 1218909, 'T'),
 (25, 1218910, 'C'),
 (26, 1218911, 'T'),
 (27, 1218912, 'C'),
 (28, 1218913, 'C'),
 (29, 1218914, 'A'),
 (30, 1218915, 'G'),
 (31, 1218916, 'G'),
 (32, 1218917, 'A'),
 (33, 1218918, 'C'),
 (34, 1218919, 'T'),
 (35, 1218920, 'T'),
 (36, 1218921, 'C'),
 (37, 1218922, 'C'),
 (38, 1218923, 'T'),
 (39, 1218924, 'G'),
 (40, 1218925, 'T'),
 (41, 1218926, 'G'),
 (42, 1218927, 'C'),
 (43, 1218928, 'T'),
 (44, 1218929, 'G'),
 (45, 1218930, 'A'),
 (46, 1218931, 'G'),
 (47, 1218932, 'A'),
 (

In [128]:
len(read_object_dict['A00711:132:HYJY2DSXX:1:1175:15935:12336'].get_reference_sequence())

144

In [115]:
read_df.iloc[45,:]['ref_sequence']

'CCTGGTACCAGCGGTCCTTCAGGTTCTCCAGGACTTCCTGTGCTGAGAGGGGCGCAGCCTGTGGGTATCGAGGCCGACAGACGCCAGCACGCAAATCCAGAAAGTTCCGAGAGGTGCTGCCTGAACTCGAGGGACACAGCCACC'

In [116]:
read_df.iloc[45,:]['meth_str']

'0000000000010000000000000000000000000000000000000000100000000000000010000010000001000000010000000000000000010000000000000000000100000000000000000000'

In [36]:
read_aligned_pairs

[(0, 1178535, 'G'),
 (1, 1178536, 'G'),
 (2, 1178537, 'G'),
 (3, 1178538, 'C'),
 (4, 1178539, 'C'),
 (5, 1178540, 'A'),
 (6, 1178541, 'C'),
 (7, 1178542, 'A'),
 (8, 1178543, 'A'),
 (9, 1178544, 'G'),
 (10, 1178545, 'A'),
 (11, 1178546, 'A'),
 (12, 1178547, 'T'),
 (13, 1178548, 'G'),
 (14, 1178549, 'A'),
 (15, 1178550, 'A'),
 (16, 1178551, 'G'),
 (17, 1178552, 'C'),
 (18, 1178553, 'T'),
 (19, 1178554, 'C'),
 (20, 1178555, 'c'),
 (21, 1178556, 'G'),
 (22, 1178557, 'T'),
 (23, 1178558, 'C'),
 (24, 1178559, 'T'),
 (25, 1178560, 'C'),
 (26, None, None),
 (27, 1178561, 'A'),
 (28, 1178562, 'A'),
 (29, 1178563, 'A'),
 (30, 1178564, 'A'),
 (31, 1178565, 'A'),
 (32, 1178566, 'A'),
 (33, 1178567, 'A'),
 (34, 1178568, 'A'),
 (35, 1178569, 'A'),
 (36, 1178570, 'A'),
 (37, 1178571, 'A'),
 (38, 1178572, 'A'),
 (39, 1178573, 'A'),
 (40, 1178574, 'A'),
 (41, 1178575, 'A'),
 (42, 1178576, 'G'),
 (43, 1178577, 'T'),
 (44, 1178578, 'T'),
 (45, 1178579, 'C'),
 (46, 1178580, 'T'),
 (47, 1178581, 'C'),
 (48

In [ ]:
test_dir = '/well/ludwig/users/dyp502/read_classifier/data/testing_data/test_bam2read/'